# NeRF_PL

## 1. Run COLMAP
Input data in `$DATASET_PATH/images`

In [ ]:
import os

os.environ['DATASET_PATH'] = "/workspace/colmap"

!colmap automatic_reconstructor \
    --workspace_path $DATASET_PATH \
    --image_path $DATASET_PATH/images

## 2. Creating a dataset in LLFF format

In [ ]:
%cd /workspace/LLFF

!python imgs2poses.py $DATASET_PATH

## 3. Train NeRF_PL 
360 inward-facing scene

In [ ]:
%cd /content/nerf_pl

# set training configurations here
os.environ['IMG_W'] = "504" # image width (do not set too large)
os.environ['IMG_H'] = "378" # image height (do not set too large)
os.environ['NUM_EPOCHS'] = "30" # number of epochs to train (depending on how many images there are,
                                # 20~30 might be enough)
os.environ['EXP'] = "exp" # name of the experience (arbitrary)

!python train.py \
   --dataset_name llff \
   --root_dir $DATASET_PATH \
   --N_importance 64 --img_wh $IMG_W $IMG_H \
   --spheric --use_disp \
   --num_epochs $NUM_EPOCHS --batch_size 1024 \
   --optimizer adam --lr 5e-4 \
   --lr_scheduler cosine \
   --exp_name $EXP

## 4. Test NeRF_PL

In [ ]:
os.environ['SCENE'] = 'test'
os.environ['CKPT_PATH'] = '/workspace/result/nerf_pl.ckpt'

!python eval.py \
   --root_dir $DATASET_PATH \
   --dataset_name llff --scene_name $SCENE \
   --img_wh $IMG_W $IMG_H --N_importance 64 --ckpt_path $CKPT_PATH

100% 120/120 [20:48<00:00, 10.41s/it]
